In [3]:
import numpy as np
from numpy.random import choice
import pandas as pd
from scipy.io import loadmat
from math import floor
import random
import copy
import json

Using TensorFlow backend.


### genome

In [0]:
!pip install genomepy --upgrade pyyaml

In [0]:
import genomepy

#get hg19 reference genome ZERO INDEXED
genomepy.install_genome("hg19", "UCSC", genome_dir="/data/genomes")
g = genomepy.Genome("hg19", genome_dir="/data/genomes")

downloading from http://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz...
done...
name: hg19
local name: hg19
fasta: /data/genomes/hg19/hg19.fa


In [0]:
#get dict of chr lens
df_chr = pd.read_csv('chr_lengths.csv')
chr_len = {}
for i in range(24):
    chrom = 'chr' + df_chr.iloc[i,0]
    chr_len[chrom] = df_chr.iloc[i,1]

#get dict of chr probs (based on len)
g_len = sum(chr_len.values())
chr_prob = {}
for key in chr_len.keys():
    chr_prob[key] = chr_len[key]/g_len

In [0]:
#mapping base pairs for switching strand
bp = {}
bp['A'] = 'T'
bp['G'] = 'C'
bp['T'] = 'A'
bp['C'] = 'G'

### functions

In [0]:
#encode sequence into tensor
def encode(df):
    n = df.shape[0]
    m = df['seq'].map(len).max()
    a = np.empty((n,m,4))
  
    d = {'A': np.array([1,0,0,0]),
         'C': np.array([0,1,0,0]),
         'G': np.array([0,0,1,0]),
         'T': np.array([0,0,0,1]), 
         'N': np.array([0,0,0,0])}

    for i in range(n):
        seq = df['seq'][i]
        extra = m-len(seq)
        if extra > 0:
            seq += 'N'*extra
    
        seq_array = list(seq)
        for j in range(m):
            base = seq_array[j]
            a[i][j] = d[base]
    
    return a

In [0]:
def get_rand_seq(gc,n,seqs):
    s = seqs[n][gc][0]
    del seqs[n][gc][0]
    return s

In [0]:
def get_gc(seq):
    count = 0
    for bp in seq:
        if bp == 'C' or bp == 'G':
        count += 1
      
    gc_bin = floor(5*count/len(seq))/5
    if gc_bin == 1:
        gc_bin = .8
    
    return gc_bin

In [0]:
def switch_strand(seq):
    out = ''
    for s in seq[::-1]: #reverse step
        out += bp[s] #match step
    return out

### generate neg sequences

In [0]:
data = pd.read_csv('all_rbp_sequences.csv')

In [0]:
#query a ton of random sequences
dirName = 'NEGATIVE/database'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    
#get lengths we need
max_len = data['sequence'].map(len).max()
min_len = data['sequence'].map(len).min()
seq_lens = [i for i in range(min_len, max_len+1)]
#seq_lens = [i for i in range(102,152)]

rand_seqs = {}
for seql in seq_lens:
    d = {}
    for j in np.linspace(0,.8,5): #buckets of 0, .2, .4, .6, .8
        d[round(j,1)] = []
  
    n = 100000
    #if seql < 130:
    #    n = 350000
  
    for k in range(n):
        #pick random chr
        chr_rand = choice(list(chr_prob.keys()),1,list(chr_prob.values()))[0]
    
        #pick random start pos
        rand_start = random.randint(1,chr_len[chr_rand])
    
        #construct sequence
        s = str(g[chr_rand][rand_start:rand_start+seql]).upper()
    
        if 'N' not in s:      
            #pick strand
            neg = random.choice([0,1])
        if neg:
            s = switch_strand(s)
      
        #add seq to proper gc
        gc = get_gc(s)
        d[gc].append((s,chr_rand,rand_start,neg))
    
    rand_seqs[seql] = d
    
file = dirName + '/' + 'neg100k.txt'
with open(file, 'w') as file:
    file.write(json.dumps(rand_seqs))

In [0]:
###ORIGINAL RUN, ONLY USING ONE SET (300K)
def get_neg_seqs(RBP):
    d_copy = copy.deepcopy(rand_seqs)
    neg_seq = []
    df = data.loc[data['RBP'] == RBP]
    for seq in df['sequence']:
        n = str(len(seq))
        gc = str(get_gc(seq))
        s = get_rand_seq(gc,n,d_copy)
        neg_seq.append(s)
  
    df_out = pd.DataFrame(neg_seq, columns=['sequence', 'chr', 'start', 'neg'])
    name = 'NEGATIVE/' + RBP + '_neg.csv'
    df_out.to_csv(name)

In [0]:
import json
with open('neg300k.txt') as json_file:  
    rand_seqs = json.load(json_file)
    
with open('neg500k-130.txt') as json_file:  
    rand_seqs2 = json.load(json_file)

In [0]:
#FOR MANUAL CASE
import json
def get_neg_seqs(RBP):
  extra = []
  
  with open('neg300k.txt') as json_file:  
      rand_seqs = json.load(json_file)
      
  with open('neg500k-130.txt') as json_file:  
      rand_seqs2 = json.load(json_file)
      
  neg_seq = []
  df = df_remaining.loc[df_remaining['RBP'] == RBP]
  for seq in df['sequence']:
    n = str(len(seq))
    gc = str(get_gc(seq))
    
    try:
      s = get_rand_seq(gc,n,rand_seqs)
    except:
      try:
        s = get_rand_seq(gc,n,rand_seqs2)
      except:
        extra.append((n,gc))

    neg_seq.append(s)
  
  df_out = pd.DataFrame(neg_seq, columns=['sequence', 'chr', 'start', 'neg'])
  name = 'NEGATIVE/' + RBP + '_neg.csv'
  df_out.to_csv(name)
  return extra

In [0]:
RBP_names = set(df_remaining['RBP'])
manual_list = []
for RBP in RBP_names:
  extra = get_neg_seqs(RBP)
  manual_list.append((RBP,extra))

In [0]:
with open('neg100k.txt') as json_file:
  rand_seqs4 = json.load(json_file)

In [0]:
####LAST 3
def get_neg_seqs_by_gc_len(l):
  extra = []
  neg_seq = []
  with open('neg350_100k.txt') as json_file:  
    rand_seqs3 = json.load(json_file)

  for tup in l:
    n = tup[0]
    gc = tup[1]
    
    try:
      s = get_rand_seq3(gc,n,rand_seqs)
      neg_seq.append(s)
    except:
      extra.append((n,gc))
  
  df_out = pd.DataFrame(neg_seq, columns=['sequence', 'chr', 'start', 'neg'])
  name = 'NEGATIVE/' + RBP + '_neg_PART2.csv'
  df_out.to_csv(name)
  
  return extra

In [0]:
last_bit = []
for tup in manual_list:
  remain = get_neg_seqs_by_gc_len(tup[1])
  if len(remain) > 0:
    last_bit.append((tup[0],remain))

In [0]:
###MAIN FUNCTION -- RUN THIS!!!
RBP_names = set(df_remaining['RBP'])
manual_list = []
for RBP in RBP_names:
  try:
    get_neg_seqs(RBP)
  except:
    manual_list.append(RBP)

In [0]:
#LAST 3 RBPs
manual_list = ['HepG2.CSTF2T', 'HepG2.FAM120A', 'HepG2.NKRF']
df_remaining = data[data['RBP'].isin(manual_list)]
del data

In [0]:
def get_neg_pass1(RBP):
  
  with open('neg300k.txt') as json_file:  
      rand_seqs = json.load(json_file)
      
  with open('neg500k-130.txt') as json_file:  
      rand_seqs2 = json.load(json_file)
      
  neg_seq, extra = [], []
  
  df = df_remaining.loc[df_remaining['RBP'] == RBP]
  for seq in df['sequence']:
    n = str(len(seq))
    gc = str(get_gc(seq))
    
    try:
      s = get_rand_seq(gc,n,rand_seqs)
      neg_seq.append(s)
    except:
      try:
        s = get_rand_seq(gc,n,rand_seqs2)
        neg_seq.append(s)
      except:
        extra.append((n,gc))
  
  df_out = pd.DataFrame(neg_seq, columns=['sequence', 'chr', 'start', 'neg'])
  name = 'NEGATIVE/' + RBP + '_neg.csv'
  df_out.to_csv(name)
  return extra

In [0]:
RBP_names = set(df_remaining['RBP'])
extras_needed = {}
for RBP in RBP_names:
    extra = get_neg_pass1(RBP)
    extras_needed[RBP] = extra

In [0]:
def get_neg_pass2(RBP,l):
  
  with open('neg350_100k.txt') as json_file:  
      rand_seqs3 = json.load(json_file)
      
  neg_seq, extra = [], []

  for tup in l:
    n = tup[0]
    gc = tup[1]
    
    try:
      s = get_rand_seq(gc,n,rand_seqs3)
      neg_seq.append(s)
    except:
      extra.append((n,gc))
  
  df_out = pd.DataFrame(neg_seq, columns=['sequence', 'chr', 'start', 'neg'])
  name = 'NEGATIVE/' + RBP + '_neg_PART2.csv'
  df_out.to_csv(name)
  return extra

In [0]:
more_extras_needed = {}
for key in extras_needed:
    extra = get_neg_pass2(key,extras_needed[key])
    if len(extra) > 0:
      more_extras_needed[key] = extra

In [0]:
def get_neg_pass3(RBP,l):
  
  with open('neg100k.txt') as json_file:  
      rand_seqs4 = json.load(json_file)
      
  neg_seq = []

  for tup in l:
    n = tup[0]
    gc = tup[1]
    s = get_rand_seq(gc,n,rand_seqs4)
    neg_seq.append(s)
  
  df_out = pd.DataFrame(neg_seq, columns=['sequence', 'chr', 'start', 'neg'])
  name = 'NEGATIVE/' + RBP + '_neg_PART3.csv'
  df_out.to_csv(name)

In [0]:
for key in more_extras_needed:
    get_neg_pass3(key,more_extras_needed[key])

In [28]:
more_extras_needed

{'HepG2.CSTF2T': [('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8'),
  ('102', '0.8')]}

In [0]:
with open('neg100k.txt') as json_file:  
    rand_seqs4 = json.load(json_file)

In [0]:
#now, combine them into one:
fam_part1 = pd.read_csv('NEGATIVE/HepG2.FAM120A_neg_PART1.csv')
fam_part2 = pd.read_csv('NEGATIVE/HepG2.FAM120A_neg_PART2.csv')
df_fam = pd.concat([fam_part1, fam_part2])
df_fam.to_csv('NEGATIVE/HepG2.FAM120A_neg.csv')


nkrf1 = pd.read_csv('NEGATIVE/HepG2.NKRF_neg_PART1.csv')
nkrf2 = pd.read_csv('NEGATIVE/HepG2.NKRF_neg_PART2.csv')
df_nkrf = pd.concat([nkrf1, nkrf2])
df_nkrf.to_csv('NEGATIVE/HepG2.NKRF_neg.csv')

cst1 = pd.read_csv('NEGATIVE/HepG2.CSTF2T_neg_PART1.csv')
cst2 = pd.read_csv('NEGATIVE/HepG2.CSTF2T_neg_PART2.csv')
cst3 = pd.read_csv('NEGATIVE/HepG2.CSTF2T_neg_PART3.csv')
df_cst = pd.concat([cst1,cst2,cst3])
df_cst.to_csv('NEGATIVE/HepG2.CSTF2T_neg.csv')

### combine and format

In [0]:
import os
a_neg_list = [0] * 160

count = 0
for filename in os.listdir('NEGATIVE'):
  full_path = 'NEGATIVE/' + filename
  df = pd.read_csv(full_path)

  df['RBP'] = filename[:-8]
  df['chr'] = df.apply(lambda row: row.chr[3:], axis=1)
  df['end'] = df.apply(lambda row: row.start + len(row.sequence) - 1, axis=1)
  df.loc[df.neg == 0, 'strand'] = '+'
  df.loc[df.neg == 1, 'strand'] = '-'

  df_out = df[['RBP','chr','start','end','strand','sequence']]  
  a_neg_list[count] = df_out
  
  count += 1

In [0]:
df_neg_all = pd.concat(a_neg_list)

In [ ]:
df_neg_all.to_csv('all_neg_seqs.csv')